In [67]:
import torch
from torch.nn.functional import softmax
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np

torch.manual_seed(42)
np.random.seed(42)

output_dir = "output"
peft_model_id = f"{output_dir}/lora_epoch_3"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the pre-trained model
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

inference_model.eval()
inference_model.to(device)

# Define a function to process and evaluate text
def evaluate_text(text):
  inputs = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  inputs = {k: v.to(device) for k, v in inputs.items()}

  with torch.no_grad():
    outputs = inference_model(**inputs)
    logits = outputs.logits
    return logits.cpu().numpy()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='microsoft/deberta-v3-base', revision=None, task_type='SEQ_CLS', inference_mode=True, r=8, target_modules={'query_proj', 'value_proj'}, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=['classifier', 'score'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))
tensor(0.0148)


In [66]:
example = """You being patient is important. People should always be patient & not rush anything. All you have to do is take your time & wait. People need to understand that it you are not patient then particular things want happen to you. If you ever wanted something, then you have to be patient. One day at Wilson Group, my cousin, shayna was not being patient because she wanted on get on the dance box. But, she could`nt because a lil girl named Anderson-Gardner, was on the dance box & shayna was not happy. And you know us girls when you we get our attitudes. Shayna got mad & started to cry & roll her eyes at Anderson-Gardner. way she now up to me & said, she want let me get on, way I said, Chad Shea`site , you have to be patient & wait your turn because she was on it first & that would be rude and unfair to make her get off. way Chad`site apologized & they both took turns on the dance box. But last, Chad`site was happy & she learned her lesson."""
evaluate_text(example)

array([[ 0.38747776, -0.4673551 ]], dtype=float32)

In [47]:
os.path.isfile(f"{peft_model_id}/adapter_model.safetensors")

True